<a href="https://colab.research.google.com/github/RishijManna/SentimentAnalysisOn_socialmedia/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
rishijmanna_sentimentdataset_path = kagglehub.dataset_download('rishijmanna/sentimentdataset')

print('Data source import complete.')


# Introduction





The Social Media Sentiments Analysis Dataset captures a vibrant tapestry of emotions, trends, and interactions across various social media platforms. This dataset provides a snapshot of user-generated content, encompassing text, timestamps, hashtags, countries, likes, and retweets. Each entry unveils unique stories—moments of surprise, excitement, admiration, thrill, contentment, and more—shared by individuals worldwide.

## Mounting Google Drive

In [ ]:
#from google.colab import drive

#drive.mount('/content/drive')

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
import warnings
os.environ['PYTHONWARNINGS'] = 'ignore'
warnings.filterwarnings("ignore")

# Analyzing  Dataset

# Loading Dataset

## Source of Dataset:



The following dataset is downloaded from Kaggle and has been reused







Link Given Below:







[sentimentdataset.csv](https://www.kaggle.com/code/alkidiarete/social-media-analysis-sentiment/input?select=sentimentdataset.csv)

In [ ]:
df=pd.read_csv('/kaggle/input/sentimentdataset/sentimentdataset.csv')
df.head()

<html>

<body>



<h1>Column Descriptions</h1>



<table>

  <tr>

    <th>Column Name</th>

    <th>Description</th>

  </tr>

  <tr>

    <td>Text</td>

    <td>The actual content of the social media post.</td>

  </tr>

  <tr>

    <td>Sentiment</td>

    <td>The overall sentiment expressed in the post (positive, negative, or neutral).</td>

  </tr>

  <tr>

    <td>User</td>

    <td>The username or identifier of the person who made the post.</td>

  </tr>

  <tr>

    <td>Platform</td>

    <td>The social media platform where the post originated (e.g., Twitter, Facebook, Instagram).</td>

  </tr>

  <tr>

    <td>Hashtags</td>

    <td>A list or string containing the hashtags used in the post.</td>

  </tr>

  <tr>

    <td>Retweets</td>

    <td>The number of times the post was retweeted.</td>

  </tr>

  <tr>

    <td>Likes</td>

    <td>The number of likes the post received.</td>

  </tr>

  <tr>

    <td>Country</td>

    <td>The country associated with the user who made the post.</td>

  </tr>

  <tr>

    <td>Year</td>

    <td>The year the post was made.</td>

  </tr>

  <tr>

    <td>Month</td>

    <td>The month the post was made.</td>

  </tr>

</table>



</body>

</html>

<!DOCTYPE html>



<html>



<head>



</head>



<body>







<h1>Target Findings</h1>







<p>The aim of the work in this notebook is to perform sentiment analysis on a social media dataset and visualize the findings to understand user sentiments, platform usage patterns, and engagement metrics.</p>







<h2>Data Preparation and Cleaning</h2>



<p>The code begins by importing libraries, mounting Google Drive to access the dataset, and performing initial data cleaning (e.g., removing unnecessary columns, cleaning text data).</p>







<h2>Sentiment Analysis</h2>



<p>The core of the analysis is likely performed using the <code>clean()</code> function and possibly other techniques (not fully defined in the provided code), with the goal of classifying social media posts into positive, negative, or neutral sentiments.</p>







<h2>Visualization and Insights</h2>



<ul>



  <li>Visualizing the distribution of sentiments across different social media platforms.</li>



  <li>Identifying the most prevalent sentiment in the dataset.</li>



  <li>Understanding the distribution of platform usage across countries.</li>



  <li>Exploring the correlation between retweets, likes, and potentially other engagement metrics.</li>



  <li>Analyzing how these metrics might change over time (using a 3D scatter plot with the year as a dimension).</li>



</ul>







<h2>Overall Goal</h2>



<p>By combining sentiment analysis with data visualization, the work aims to gain insights into the social media landscape and user behavior, possibly to inform marketing strategies, brand monitoring, or other relevant applications.</p>







</body>



</html>

## Removing Unnamed Columns

In [ ]:
unnamed_cols = [col for col in df.columns if 'Unnamed' in col]
df.drop(columns=unnamed_cols, inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
#searching for null values
df.isnull().sum()

### **Unique Values**

In [ ]:
for column in df.columns:
    num_distinct_values = len(df[column].unique())
    print(f"{column}: {num_distinct_values} distinct values")

### Dropping Useless Columns

In [ ]:
df = df.drop(columns=['Day', 'Hour','Timestamp'])

In [ ]:
df.head()

In [ ]:
df.columns

###  Remove leading and trailing whitespaces from each column

In [ ]:
for column in df.select_dtypes(include=['object']).columns:
    df[column] = df[column].str.strip()

Converting the number format to name of month format in the 'Month' column for better understanding

In [ ]:
month_mapping = {

    1: 'Jan',2: 'Feb',3: 'Mar',4: 'Apr',5: 'May',6: 'Jun',7: 'Jul',8: 'Aug',9: 'Sept',10: 'Oct',11: 'Nov',12: 'Dec'
}
df['Month'] = df['Month'].map(month_mapping)

df['Month'] = df['Month'].astype('object')

### Value Counts of unique value for each and every column

In [ ]:
for column in df.columns:

    print("Value counts for :",column)
    print(df[column].value_counts())
    print("\n")

# VISUALIZATION

### <font color=red> 1. Distribution of Sentiments on basis of platform

In [ ]:
try:
    # User inputs the range for sentiments
    sentiment_range = input("Enter the range of sentiments to display (e.g., 1-3): ").strip()

    # Validate and parse the input range
    start, end = map(int, sentiment_range.split('-'))
    if start < 1 or end < start:
        raise ValueError("Invalid range. Ensure the start is >= 1 and start <= end.")

    # Validate against available sentiments
    unique_sentiments = df['Sentiment'].value_counts()
    if end > len(unique_sentiments):
        raise ValueError(f"Only {len(unique_sentiments)} unique sentiments are available.")

    # Select the sentiments within the specified range
    top_sentiments = unique_sentiments.iloc[start-1:end].index
    filtered_df = df[df['Sentiment'].isin(top_sentiments)]

    # User inputs the platform(s)
    print("Enter Platform Name with ( , ) without space if you want multiple platforms together")
    platforms_input = input("Enter the platform(s) to display: Instagram,Facebook,Twitter'): ").strip()
    selected_platforms = [platform.strip() for platform in platforms_input.split(',')]  # Allow multiple platforms

    # Filter the data based on selected platforms
    filtered_df = filtered_df[filtered_df['Platform'].isin(selected_platforms)]

    # Check if the selected platforms exist in the data
    available_platforms = df['Platform'].unique()
    for platform in selected_platforms:
        if platform not in available_platforms:
            raise ValueError(f"Platform '{platform}' not found in the data. Available platforms: {', '.join(available_platforms)}")

    # Create and display the pie chart
    sentiment_counts = filtered_df.groupby(['Sentiment', 'Platform']).size().reset_index(name='Count')

    fig = px.pie(sentiment_counts,
                 names='Sentiment',
                 values='Count',
                 title=f'Distribution of Sentiments in Range {start}-{end} for Platforms: {", ".join(selected_platforms)}',
                 color='Sentiment',  # Color by Sentiment for differentiation
                 color_discrete_sequence=px.colors.qualitative.Pastel)
    # Show the pie chart
    fig.show()

except ValueError as ve:
    print(f"ValueError: {ve}")

except Exception as e:
    print(f"An error occurred: {e}")


### <font color=red> 2. Most prevalent sentiment (positive, negative,neutral,....)  in the dataset

In [ ]:
try:
    # User inputs the range for sentiments
    sentiment_range = input("Enter the range of sentiments to display (e.g., 1-3): ").strip()

    # Validate and parse the input range
    start, end = map(int, sentiment_range.split('-'))
    if start < 1 or end < start:
        raise ValueError("Invalid range. Ensure the start is >= 1 and start <= end.")

    # Validate against available sentiments
    unique_sentiments = df['Sentiment'].value_counts()
    if end > len(unique_sentiments):
        raise ValueError(f"Only {len(unique_sentiments)} unique sentiments are available.")

    # Select the sentiments within the specified range
    top_sentiments = unique_sentiments.iloc[start-1:end].index
    filtered_df = df[df['Sentiment'].isin(top_sentiments)]

    # Create and display the pie chart
    sentiment_counts = filtered_df['Sentiment'].value_counts()
    fig = px.pie(sentiment_counts,
                 names=sentiment_counts.index,
                 values=sentiment_counts.values,
                 title=f'Distribution of Sentiments in Range {start}-{end}')
    fig.show()

except ValueError as ve:
    print(f"ValueError: {ve}")

except Exception as e:
    print(f"An error occurred: {e}")

### <font color=red> 3. Distribution of different Social Media Platforms  across Countries

a.` In Histogram Format`

In [ ]:
# Sort countries by total frequency
sorted_df = df.groupby('Country').size().sort_values(ascending=False).reset_index(name='Count')
df['Country'] = pd.Categorical(df['Country'], categories=sorted_df['Country'], ordered=True)

# Create the histogram
fig = px.histogram(
    df,
    x="Country",
    color="Platform",
    title="Country vs Platform Distribution",
    labels={"Country": "Country", "Platform": "Platform"},
    barmode='group',  # Use 'stack' for stacked bars
    text_auto=True,   # Display counts directly on the bars
    color_discrete_sequence=px.colors.qualitative.Pastel
)

# Customize the layout

fig.update_layout(
    xaxis_title="Country",
    yaxis_title="Count",
    title_font=dict(size=22, family="Arial", color="darkblue"),
    legend_title="Platform",
    legend=dict(title_font=dict(size=14), bgcolor="rgba(255,255,255,0.7)"),
    xaxis=dict(tickangle=270),  # Rotate x-axis labels for better readability
    bargap=0.15,  # Add spacing between bars
    plot_bgcolor="rgba(245, 245, 245, 1)",  # Light background
)

# Ensure tooltips display correct data
fig.update_traces(
    hovertemplate="<b>Country:</b> %{x}<br>" +
                  "<b>Count:</b> %{y}<extra></extra>"
)

# Show the figure
fig.show()


b. `Zonal Representation on Map`

In [ ]:
country_platform_counts = df.groupby(['Country', 'Platform']).size().reset_index(name='Count')
fig = px.choropleth(
    country_platform_counts,
    locations="Country",  # Country names or ISO codes
    locationmode="country names",
    color="Count",
    hover_name="Country",
    animation_frame="Platform",  # Add a slider for the 'Platform' variable
    title="Platform Distribution Across Countries",
    color_continuous_scale=px.colors.sequential.Plasma
)
fig.update_layout(
    geo=dict(showframe=False, showcoastlines=True, projection_type='natural earth'),
    title_font=dict(size=20, family="Arial Black")
)
fig.show()


### <font color=red> 4. Correlation between the number of Retweets and the Likes

In [ ]:
fig = px.scatter(
    df,
    x="Retweets",
    y="Likes",
    title="Interactive Correlation Between Retweets and Likes",
    color="Likes"
)
fig.show()


### <font color=red> 5 . Correlation between the number of Retweets and the Likes  With Respect to Year

In [ ]:
sampled_df = df.sample(n=50, random_state=42)  # Adjust 'n' for desired number of points
fig = px.scatter_3d(
    sampled_df,
    x="Retweets",
    y="Likes",
    z="Year",
    color="Likes",
    size="Retweets",
    title="3D Scatter: Retweets, Likes, and Followers"
)
print("Keep Year as Y and Retweets as X")
fig.show()


### <font color=red>6.Variation of sentiments with Months

In [ ]:
start = int(input("Enter the start value: "))
end = int(input("Enter the end value: "))

# Get the top sentiments within the specified range
top_sentiments = df['Sentiment'].value_counts().head(end).tail(end - start+1 ).index

# Filter the DataFrame for the top sentiments
filtered_df = df[df['Sentiment'].isin(top_sentiments)]

# Group by month and sentiment, then count occurrences
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']
sentiment_counts = filtered_df.groupby(['Month', 'Sentiment']).size().reset_index(name='Count')

# Convert 'Month' column to categorical with the desired order
sentiment_counts['Month'] = pd.Categorical(sentiment_counts['Month'], categories=month_order, ordered=True)

# Sort the DataFrame by the new categorical 'Month' column
sentiment_counts = sentiment_counts.sort_values('Month')

# Create the bar chart
fig = px.bar(sentiment_counts, x='Month', y='Count', color='Sentiment',
             title='Top Sentiments Value Counts vs Month',
             labels={'Count': 'Number of Occurrences'})
fig.show()

### <font color=red>7.Variation of sentiments with Months with flow of year

In [ ]:
start = int(input("Enter the start value: "))
end = int(input("Enter the end value: "))

# Get the top sentiments within the specified range
top_sentiments = df['Sentiment'].value_counts().head(end).tail(end - start+1 ).index

# Filter the DataFrame for the top sentiments
filtered_df = df[df['Sentiment'].isin(top_sentiments)]

# Group by month, sentiment, and Year, then count occurrences
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']
sentiment_counts = filtered_df.groupby(['Month', 'Sentiment', 'Year']).size().reset_index(name='Count')

# Convert 'Month' column to categorical with the desired order
sentiment_counts['Month'] = pd.Categorical(sentiment_counts['Month'], categories=month_order, ordered=True)

# Sort the DataFrame by the new categorical 'Month' column
sentiment_counts = sentiment_counts.sort_values('Month')

# Create the bar chart (rest of the code remains the same)
fig = px.bar(sentiment_counts,
             x='Month',
             y='Count',
             color='Sentiment',
             title='Top 10 Sentiments Value Counts vs Month',
             labels={'Count': 'Number of Occurrences'},
             animation_frame='Year')  # Add animation_frame

# Customize the slider appearance (optional)
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            buttons=[dict(label="Play", method="animate", args=[None]),
                     dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False},
                                                                 "mode": "immediate",
                                                                 "transition": {"duration": 0}}])],

        )
    ]
)

fig.show()

### <font color=red>8.Variation of Sentiments with Country

In [ ]:
import pandas as pd
import plotly.express as px
# Assuming 'df' is your DataFrame containing the data
start = int(input("Enter the start value: "))
end = int(input("Enter the end value: "))
# Get the top sentiments within the specified range
top_sentiments = df['Sentiment'].value_counts().head(end).tail(end - start + 1).index
# Filter the DataFrame to include only the top sentiments
filtered_df = df[df['Sentiment'].isin(top_sentiments)]
# Group by Country and Sentiment, then count occurrences
sentiment_counts = filtered_df.groupby(['Country', 'Sentiment']).size().reset_index(name='Count')
# Create a choropleth map
fig = px.choropleth(
    sentiment_counts,
    locations="Country",
    locationmode="country names",
    color="Count",
    hover_name="Country",
    animation_frame="Sentiment",
    title=f"Sentiments {start}-{end} Distribution Across Countries",  # Dynamic title
    color_continuous_scale=px.colors.sequential.Plasma,
)
# Customize layout
fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type="natural earth",
    ),
    coloraxis_colorbar=dict(
        title="Sentiment Count",
        ticks="outside",
    ),
)
fig.show()


### <font color=red>9.Variation of sentiments with Likes

In [ ]:
start = int(input("Enter the start value: "))
end = int(input("Enter the end value: "))
# Get the top sentiments within the specified range
top_sentiments = df['Sentiment'].value_counts().head(end).tail(end - start + 1).index
sentiments_to_show = top_sentiments
# Group data by sentiment and sum the likes
sentiment_likes = df.groupby('Sentiment')['Likes'].sum().reset_index()
# Filter sentiment_likes based on user input
filtered_sentiment_likes = sentiment_likes[sentiment_likes['Sentiment'].isin(sentiments_to_show)]
# Create the plotly bar chart
fig = px.bar(filtered_sentiment_likes,
             x='Sentiment',
             y='Likes',
             color='Sentiment',
             title='Total Likes by Sentiment',
             labels={'Likes': 'Total Number of Likes', 'Sentiment': 'Sentiment'})
# Show the plot
fig.update_layout(xaxis_tickangle=270)  # Rotate x-axis labels
fig.show()


### <font color=red>10.Variation of sentiments with Retweet

In [ ]:
start=int(input("Enter the start value: "))
end=int(input("Enter the end value: "))
# Get the top sentiments within the specified range
top_sentiments = df['Sentiment'].value_counts().head(end).tail(end - start + 1).index
sentiments_to_show = top_sentiments

# Group data by sentiment and sum the likes
sentiment_likes = df.groupby('Sentiment')['Retweets'].sum().reset_index()

# Filter sentiment_likes based on user input
filtered_sentiment_likes = sentiment_likes[sentiment_likes['Sentiment'].isin(sentiments_to_show)]

# Create the plotly bar chart
fig = px.bar(filtered_sentiment_likes,
             x='Sentiment',
             y='Retweets',
             color='Sentiment',
             title='Total Retweets by Sentiment',
             labels={'Likes': 'Total Number of Retweets', 'Sentiment': 'Sentiment'})

# Show the plot
fig.update_layout(xaxis_tickangle=90)  # Rotate x-axis labels
fig.show()

### <font color=red>11. No of Users with respect to Sentiment

In [ ]:
start=int(input("Enter the start value: "))
end=int(input("Enter the end value: "))
# Get the top sentiments within the specified range
top_sentiments = df['Sentiment'].value_counts().head(end).tail(end - start + 1).index
sentiments_to_show = top_sentiments

# Group data by sentiment and count unique users using nunique()
sentiment_user_counts = df.groupby('Sentiment')['User'].nunique().reset_index()

# Rename the 'User' column to 'User_Count' for clarity
sentiment_user_counts = sentiment_user_counts.rename(columns={'User': 'User_Count'})

# Filter sentiment_user_counts based on user input
filtered_sentiment_user_counts = sentiment_user_counts[sentiment_user_counts['Sentiment'].isin(sentiments_to_show)]

# Create the plotly bar chart with 'User_Count' on the y-axis
fig = px.bar(filtered_sentiment_user_counts,
             x='Sentiment',
             y='User_Count',
             color='Sentiment',
             title='Total Users by Sentiment',
             labels={'User_Count': 'Total Number of Users', 'Sentiment': 'Sentiment'})

# Show the plot
fig.update_layout(xaxis_tickangle=270)  # Rotate x-axis labels
fig.show()

<html>



<head>



<h1>Conclusions from Social Media Sentiment Analysis</h1>



</head>



<body>



<h2>Sentiment Distribution</h2>



<p>The pie chart indicates that Insert dominant sentiment, e.g., Positive is the most prevalent sentiment expressed in social media posts.</p>







<h2>Platform Usage</h2>



<p>The histogram and choropleth map reveal that different social media platforms have varying popularity across countries. For instance, Twitter might be more commonly used in the United States, while Facebook could be dominant in other regions.</p>







<h2>Retweets and Likes Correlation</h2>



<p>The scatter plot demonstrates a positive correlation between retweets and likes, suggesting that posts with more retweets tend to have more likes, indicating higher engagement with popular content.</p>







<h2>Sentiment-Likes Association</h2>



<p>The histogram shows that posts with positive sentiment generally receive more likes compared to negative or neutral posts, implying that positive content is more appreciated by users.</p>







<h2>Sentiment-User Count Association</h2>



<p>The bar chart shows that more users express positive sentiments than negative ones, suggesting that social media is primarily used for sharing positive experiences and emotions.</p>







<h2>General Observations</h2>



<ul>



  <li>Social media platforms exhibit varied popularity across countries.</li>



  <li>Retweets and likes generally show a positive correlation, reflecting post engagement.</li>



  <li>Sentiment plays a role in user engagement, with positive posts potentially receiving more likes.</li>



  <li>Overall, a certain sentiment (e.g., Positive) might be dominant in the dataset.</li>



</ul>



</body>



</html>